In [ ]:
"""
Module Name: HMC.py

Description:
    Making a list of distributions and functions for use in numerical sampling and analysis of approximate convergence. 

Author: John Gallagher
Created: 2025-06-03
Last Modified: 
Version: 1.0.0

Dependencies:
    - numpy
    - scipy
    - matplotlib.pyplot
"""
import numpy as np
from scipy.stats import norm, uniform, expon
import matplotlib.pyplot as plt
np.random.seed(1234)  # For reproducibility

In [3]:
def nongauss_f(x):
    """Non-gaussian target distribution."""
    return np.exp(-x**2 * (2 + np.sin(5*x) + np.sin(2*x)))

def gauss_f(x):
    """Gaussian target distribution."""
    return np.exp(-x**2)

In [ ]:
def metropolis_step(x, sigma, function):
    """ Perform a single Metropolis-Hastings step.
    Args:
        x (float): Current state of the chain.
        sigma (float): Step size for the proposal distribution.
        function (callable): Target distribution function.
    
    Returns:  
        tuple: A tuple containing the new state and a boolean indicating whether the step was accepted.
    """

    proposed_x = np.random.normal(x, sigma)
    alpha = min(1, function(proposed_x)/function(x)) #why does this fraction work? Base case is we draw from the center of distributions and we want more information on the curvature.  The smaller probabilties (lower values) are associated with getting more information about the smaller regions. 
    u = np.random.uniform()
    if u < alpha:
        value = proposed_x
        accepted = True
    else:
        value = x
        accepted = False
    return value, accepted
def metropolis_sampler(initial_val, function, n=1000, sigma = 1):
    """
    Perform Metropolis-Hastings sampling.

    Args:
        initial_val (float): Initial value for the chain.
        function (callable): Target distribution function.
        n (int): Number of samples to generate.
        sigma (float): Step size for the proposal distribution.
    
    Returns:
        list: List of tuples containing sampled values and acceptance status.
    """

    results = []
    current_state = initial_val
    for i in range(0, n):
        out = metropolis_step(current_state, sigma, function)
        current_state = out[0]
        results.append(out)
    return results

def plot_results(self, x_range=(-11, 11), n_points=10000, bins=50):
    """
    Plot the sampling results against the target distribution.
    
    Args:
        x_range (tuple): Range of x values for plotting
        n_points (int): Number of points for target function curve
        bins (int): Number of histogram bins
    """
    if self.samples is None:
        raise ValueError("No samples available. Run sample() first.")
        
    fig, ax1 = plt.subplots()
    
    # Plot histogram of accepted samples
    ax1.hist(self.accepted['value'], bins=bins)
    ax1.set_ylabel('Frequency')
    ax1.set_xlabel('x-Value')
    ax1.set_title('Resulting Accepted Distribution vs Target Function')
    
    # Plot target distribution
    ax2 = ax1.twinx()
    x_vals = np.linspace(x_range[0], x_range[1], n_points)
    y_vals = [self.target_function(x) for x in x_vals]
    ax2.plot(x_vals, y_vals, 'r-', label='Target Function')
    ax2.set_ylabel('Target Function Value')
    ax2.set_ylim(0, 1)
    
    plt.show()

In [ ]:
n = 100000
sigma = 12
sample = metropolis_sampler(0.1, nongauss_f, n = n,sigma=sigma)

In [ ]:
sampler = MetropolisSampler(target_function=nongauss_f, sigma=12, seed=1234)
samples = sampler.sample(initial_value=0.1, n_samples=100000)
sampler.plot_results()

In [ ]:
       
def leapfrog_step(self, t,  q, p, dt, sigma, function):
    """
    Perform a single Leap-Frog step for symplectic integration.

    Args:
        t (float): Current time.
        dt (float): Time step for the integration.
        q (np.ndarray): Current state position vector.
        p (np.ndarray): Current state momentum vector.
        sigma (float): Standard deviation.
        function (callable): Target distribution function.

    Returns:
        tuple: A tuple containing the new state after the Symplectic Euler step.
    """
    # Kinetic energy term
    K = lambda t, x: 0.5 * 1/sigma**2 * np.eye(len(x))  
    # Potential energy term
    V = lambda t, x: -np.log(function(x))
    # Half step for position
    q_half = q + 0.5 * dt * K(t, x)
    # Full step for momentum
    p_full = p + dt * V(t, q_half)
    # Full step for position
    q_full = q_half + 0.5 * dt * K(t, p_full)
    return q_full, p_full

def Hamiltonian(q, p, sigma=1):
    """
    Compute the Hamiltonian of a system given position and momentum.

    Args:
        q (np.ndarray): Position vector.
        p (np.ndarray): Momentum vector.
        sigma (float): Standard deviation.

    Returns:
        float: The Hamiltonian value.
    """
    # Assuming M = sigma^2 * I, where I is the identity matrix
    sigma = 1  # This can be adjusted as needed
    kinetic_energy = 0.5  * np.dot(p, p / (sigma ** 2)) 
    potential_energy = -np.sum(np.log(q))  # Assuming q is the position vector
    return kinetic_energy + potential_energy

def HMC_step(t=1, y, sigma, function):
    """
    Perform a single Hamiltonian Monte Carlo step.

    Args:
        x (float): Current state of the chain.
        sigma (float): Step size for the proposal distribution.
        function (callable): Target distribution function.

    Returns:
        tuple: A tuple containing the new state and a boolean indicating whether the step was accepted.
    """
    q_initial = y
    p_initial = np.random.normal(np.zeros(len(y)), sigma)
    q_proposed, p_proposed = leapfrog_step(t, q_initial, 0.1, sigma, function)
    alpha = min(1, np.exp(-Hamiltonian(q_proposted, p_proposed, sigma) - Hamiltonian(q_initial, p_initial, sigma))
    u = np.random.uniform()
    if u < alpha:
        value = proposed_x
        accepted = True
    else:
        value = y
        accepted = False
    return value, accepted

def HMC_sampler(initial_val, function, n=1000, sigma=1):
    """
    Perform Hamiltonian Monte Carlo sampling.

    Args:
        initial_val (float): Initial value for the chain.
        function (callable): Target distribution function.
        n (int): Number of samples to generate.
        sigma (float): Step size for the proposal distribution.

    Returns:
        list: List of tuples containing sampled values and acceptance status.
    """
    results = []
    current_state = initial_val
    for i in range(n):
        out = HMC_step(current_state, sigma, function)
        current_state = out[0]
        results.append(out)
    return results